# Car Detection with YOLOv11 and YOLOv12 - Google Colab Version

**Dataset:** Car Detection Dataset (BMW-X5-M, Volvo-XC40, Jaguar)

**Models:** YOLOv11 and YOLOv12

**Environment:** Google Colab


## Environment Setup

In [ ]:
# Install required packages
print("Installing required packages...")
!pip install -q ultralytics roboflow
!pip install -q opencv-python-headless
!pip install -q matplotlib
!pip install -q numpy
!pip install -q seaborn
!pip install -q pandas
print("\n Packages installed successfully!")

In [ ]:
# Mount Google Drive (optional - for saving results)
from google.colab import drive

drive.mount('/content/drive')
print("\n Google Drive mounted at /content/drive")

# Create results directory in Drive
import os
DRIVE_RESULTS = '/content/drive/MyDrive/YOLO_Car_Detection_Results'
os.makedirs(DRIVE_RESULTS, exist_ok=True)
print(f" Results will be saved to: {DRIVE_RESULTS}")

In [ ]:
# Check Python and system information
import sys
import platform

print("="*80)
print("SYSTEM INFORMATION")
print("="*80)
print(f"Python version: {sys.version}")
print(f"Platform: {platform.system()} {platform.release()}")
print(f"Machine: {platform.machine()}")
print("="*80)

In [ ]:
# Import required libraries
import os
import time
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import yaml
from collections import Counter
import json
from PIL import Image
import shutil
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Deep Learning libraries
from ultralytics import YOLO
import torch

# Set visualization style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# For better plot display in Colab
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("\n" + "="*80)
print("LIBRARY VERSIONS")
print("="*80)
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"OpenCV version: {cv2.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")
print("="*80)

# Check GPU availability
print("\n" + "="*80)
print("GPU INFORMATION")
print("="*80)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
else:
    print("  No GPU detected - Training will use CPU (slower)")
    print("   Please enable GPU: Runtime > Change runtime type > GPU")
print("="*80)

print("\n All libraries imported successfully!")

In [ ]:
# Record start time for entire execution
execution_start_time = time.time()
print(f"Execution started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Dataset Loading


In [ ]:
# Configuration
DATASET_DIR = "/content/dataset" 
RESULTS_DIR = "/content/results"

# Create directories
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"Dataset directory: {DATASET_DIR}")
print(f"Results directory: {RESULTS_DIR}")

In [ ]:
# Verify dataset structure
print("\n" + "="*80)
print("VERIFYING DATASET STRUCTURE")
print("="*80)

dataset_path = DATASET_DIR

# Required paths
required_paths = [
    f'{dataset_path}/data.yaml',
    f'{dataset_path}/train/images',
    f'{dataset_path}/train/labels',
    f'{dataset_path}/valid/images',
    f'{dataset_path}/valid/labels'
]

all_paths_exist = True
for path in required_paths:
    if os.path.exists(path):
        print(f"Found: {path}")
    else:
        print(f"Missing: {path}")
        all_paths_exist = False

if not all_paths_exist:
    raise FileNotFoundError("Required dataset structure not found!")
    
print("\n Dataset structure verified!")

In [ ]:
# Load dataset configuration
with open(f'{dataset_path}/data.yaml', 'r') as f:
    data_config = yaml.safe_load(f)

print("\n" + "="*80)
print("DATASET CONFIGURATION")
print("="*80)
print(f"Number of classes: {data_config['nc']}")
print(f"\nClass names:")
for i, name in enumerate(data_config['names']):
    print(f"  {i}: {name}")
print(f"\nTrain path: {data_config.get('train', 'Not specified')}")
print(f"Validation path: {data_config.get('val', data_config.get('valid', 'Not specified'))}")
print(f"Test path: {data_config.get('test', 'Not provided')}")
print("="*80)

## Exploratory Data Analysis (EDA)

In [ ]:
def analyze_dataset_split(base_path, split='train'):
    """Analyze a dataset split and return statistics"""
    images_path = Path(base_path) / split / 'images'
    labels_path = Path(base_path) / split / 'labels'
    
    # Count images and labels
    images = list(images_path.glob('*.jpg')) + list(images_path.glob('*.png'))
    labels = list(labels_path.glob('*.txt'))
    
    print(f"\n{'='*70}")
    print(f"{split.upper()} SET STATISTICS")
    print(f"{'='*70}")
    print(f"Number of images: {len(images)}")
    print(f"Number of labels: {len(labels)}")
    
    # Analyze image dimensions (sample)
    image_dims = []
    for img_path in images[:100]:
        img = Image.open(img_path)
        image_dims.append(img.size)
    
    if image_dims:
        widths, heights = zip(*image_dims)
        print(f"\nImage Dimensions (sample of {len(image_dims)}):")
        print(f"  Width  - Min: {min(widths):4d}, Max: {max(widths):4d}, Mean: {np.mean(widths):6.1f}")
        print(f"  Height - Min: {min(heights):4d}, Max: {max(heights):4d}, Mean: {np.mean(heights):6.1f}")
    
    # Analyze class distribution
    class_counts = Counter()
    bbox_counts = []
    bbox_areas = []
    
    for label_file in labels:
        with open(label_file, 'r') as f:
            lines = f.readlines()
            bbox_counts.append(len(lines))
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    class_counts[class_id] += 1
                    
                    # Get bbox area
                    width, height = float(parts[3]), float(parts[4])
                    bbox_areas.append(width * height)
    
    if bbox_counts:
        print(f"\nBounding Box Statistics:")
        print(f"  Total boxes: {sum(bbox_counts)}")
        print(f"  Avg boxes per image: {np.mean(bbox_counts):.2f}")
        print(f"  Min/Max boxes: {min(bbox_counts)}/{max(bbox_counts)}")
    
    return {
        'class_counts': class_counts,
        'bbox_counts': bbox_counts,
        'image_dims': image_dims,
        'bbox_areas': bbox_areas,
        'images': images,
        'labels': labels
    }

# Analyze all splits
train_stats = analyze_dataset_split(dataset_path, 'train')
val_stats = analyze_dataset_split(dataset_path, 'valid')

if os.path.exists(Path(dataset_path) / 'test'):
    test_stats = analyze_dataset_split(dataset_path, 'test')
else:
    test_stats = None

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Training set class distribution
classes = [data_config['names'][i] for i in sorted(train_stats['class_counts'].keys())]
counts = [train_stats['class_counts'][i] for i in sorted(train_stats['class_counts'].keys())]

axes[0].bar(range(len(classes)), counts, color='steelblue', alpha=0.8, edgecolor='black')
axes[0].set_xlabel('Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Number of Instances', fontsize=12, fontweight='bold')
axes[0].set_title('Class Distribution - Training Set', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(len(classes)))
axes[0].set_xticklabels(classes, rotation=45, ha='right')
axes[0].grid(axis='y', alpha=0.3)

# Objects per image distribution
axes[1].hist(train_stats['bbox_counts'], bins=30, color='coral', alpha=0.8, edgecolor='black')
axes[1].set_xlabel('Number of Objects per Image', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequency', fontsize=12, fontweight='bold')
axes[1].set_title('Objects per Image Distribution', fontsize=14, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/eda_class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n Saved: {RESULTS_DIR}/eda_class_distribution.png")

In [ ]:
# Visualize sample images with annotations
def visualize_samples(num_samples=9):
    train_images_path = Path(dataset_path) / 'train' / 'images'
    train_labels_path = Path(dataset_path) / 'train' / 'labels'
    
    sample_images = np.random.choice(train_stats['images'], 
                                     min(num_samples, len(train_stats['images'])), 
                                     replace=False)
    
    grid_size = int(np.ceil(np.sqrt(num_samples)))
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(20, 20))
    axes = axes.ravel() if num_samples > 1 else [axes]
    
    for idx, img_path in enumerate(sample_images):
        # Read image
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        # Read labels
        label_path = train_labels_path / f"{img_path.stem}.txt"
        num_boxes = 0
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        x_center, y_center, width, height = map(float, parts[1:5])
                        
                        # Convert to pixel coordinates
                        x1 = int((x_center - width/2) * w)
                        y1 = int((y_center - height/2) * h)
                        x2 = int((x_center + width/2) * w)
                        y2 = int((y_center + height/2) * h)
                        
                        # Draw box
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
                        # Add label
                        class_name = data_config['names'][class_id]
                        cv2.putText(img, class_name, (x1, y1-10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        num_boxes += 1
        
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f"Sample {idx+1} ({num_boxes} objects)", fontsize=10, fontweight='bold')
    
    # Hide unused subplots
    for idx in range(len(sample_images), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/sample_annotated_images.png', dpi=300, bbox_inches='tight')
    plt.show()
    print(f"\n Saved: {RESULTS_DIR}/sample_annotated_images.png")

visualize_samples(9)

## Model Training - YOLOv11

In [ ]:
# Training parameters
EPOCHS = 50
IMG_SIZE = 640
BATCH_SIZE = 16
DEVICE = 0 if torch.cuda.is_available() else 'cpu'

print(f"Training Configuration:")
print(f"Epochs: {EPOCHS}")
print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# Train YOLOv11
print("\n" + "="*80)
print("TRAINING YOLOv11")
print("="*80)

yolov11_start_time = time.time()

# Initialize model
print("\nInitializing YOLOv11 model...")
model_v11 = YOLO('yolo11n.pt')  # Will auto-download if not present

# Train
print("\nStarting training...")
results_v11 = model_v11.train(
    data=f'{dataset_path}/data.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name='yolov11_car_detection',
    project='/content/runs/detect',
    patience=10,
    save=True,
    device=DEVICE,
    optimizer='AdamW',
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    plots=True,
    verbose=True,
    workers=2  # Reduced for Colab stability
)

yolov11_training_time = time.time() - yolov11_start_time

print(f"\n" + "="*80)
print(f"YOLOv11 Training completed in {yolov11_training_time:.2f} seconds ({yolov11_training_time/60:.2f} minutes)")
print("="*80)

In [ ]:
# Validate YOLOv11
print("\nValidating YOLOv11 model...")
val_results_v11 = model_v11.val(data=f'{dataset_path}/data.yaml', workers=2)

print("\n" + "="*80)
print("YOLOv11 VALIDATION METRICS")
print("="*80)
print(f"mAP@0.5      : {val_results_v11.box.map50:.4f}")
print(f"mAP@0.5:0.95 : {val_results_v11.box.map:.4f}")
print(f"Precision    : {val_results_v11.box.mp:.4f}")
print(f"Recall       : {val_results_v11.box.mr:.4f}")
print("="*80)

## Model Training - YOLOv12

In [ ]:
# Train YOLOv12
print("\n" + "="*80)
print("TRAINING YOLOv12")
print("="*80)

yolov12_start_time = time.time()

# Initialize model
print("\nInitializing YOLOv12 model...")
model_v12 = YOLO('yolo12n.pt')  # Will auto-download if not present

# Train
print("\nStarting training...")
results_v12 = model_v12.train(
    data=f'{dataset_path}/data.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name='yolov12_car_detection',
    project='/content/runs/detect',
    patience=10,
    save=True,
    device=DEVICE,
    optimizer='AdamW',
    lr0=0.01,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    plots=True,
    verbose=True,
    workers=2
)

yolov12_training_time = time.time() - yolov12_start_time

print(f"\n" + "="*80)
print(f"YOLOv12 Training completed in {yolov12_training_time:.2f} seconds ({yolov12_training_time/60:.2f} minutes)")
print("="*80)

In [ ]:
# Validate YOLOv12
print("\nValidating YOLOv12 model...")
val_results_v12 = model_v12.val(data=f'{dataset_path}/data.yaml', workers=2)

print("\n" + "="*80)
print("YOLOv12 VALIDATION METRICS")
print("="*80)
print(f"mAP@0.5      : {val_results_v12.box.map50:.4f}")
print(f"mAP@0.5:0.95 : {val_results_v12.box.map:.4f}")
print(f"Precision    : {val_results_v12.box.mp:.4f}")
print(f"Recall       : {val_results_v12.box.mr:.4f}")
print("="*80)

## Performance Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': ['YOLOv11', 'YOLOv12'],
    'mAP@0.5': [val_results_v11.box.map50, val_results_v12.box.map50],
    'mAP@0.5:0.95': [val_results_v11.box.map, val_results_v12.box.map],
    'Precision': [val_results_v11.box.mp, val_results_v12.box.mp],
    'Recall': [val_results_v11.box.mr, val_results_v12.box.mr],
    'Training Time (min)': [yolov11_training_time/60, yolov12_training_time/60]})

# Calculate F1 score
comparison_df['F1 Score'] = 2 * (comparison_df['Precision'] * comparison_df['Recall']) / \
                            (comparison_df['Precision'] + comparison_df['Recall'])

print("\n" + "="*80)
print("PERFORMANCE COMPARISON: YOLOv11 vs YOLOv12")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)

# Save comparison
comparison_df.to_csv(f'{RESULTS_DIR}/model_comparison_colab.csv', index=False)
print(f"\n Saved: {RESULTS_DIR}/model_comparison_colab.csv")

In [ ]:
# Visualize performance comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

metrics = ['mAP@0.5', 'mAP@0.5:0.95', 'Precision', 'Recall', 'F1 Score']
colors = ['#3498db', '#e74c3c']

for idx, metric in enumerate(metrics):
    ax = axes[idx]
    values = comparison_df[metric].values
    bars = ax.bar(comparison_df['Model'], values, color=colors, alpha=0.8, 
                  edgecolor='black', linewidth=2)
    
    ax.set_ylabel(metric, fontsize=12, fontweight='bold')
    ax.set_title(f'{metric} Comparison', fontsize=14, fontweight='bold')
    ax.set_ylim([0, 1.0])
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
               f'{height:.4f}',
               ha='center', va='bottom', fontsize=11, fontweight='bold')

# Training time comparison
ax = axes[5]
training_times = comparison_df['Training Time (min)'].values
bars = ax.bar(comparison_df['Model'], training_times, color=colors, alpha=0.8,
             edgecolor='black', linewidth=2)

ax.set_ylabel('Training Time (minutes)', fontsize=12, fontweight='bold')
ax.set_title('Training Time Comparison', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{height:.2f} min',
           ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/performance_comparison_colab.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n Saved: {RESULTS_DIR}/performance_comparison_colab.png")

## Model Evaluation

In [ ]:
# Display confusion matrices
import glob

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# YOLOv11 confusion matrix
v11_runs = sorted(glob.glob('/content/runs/detect/yolov11_*'), key=os.path.getmtime, reverse=True)
v11_cm_path = None
if v11_runs:
    for run_dir in v11_runs:
        cm_path = os.path.join(run_dir, 'confusion_matrix.png')
        if os.path.exists(cm_path):
            v11_cm_path = cm_path
            break

if v11_cm_path and os.path.exists(v11_cm_path):
    img_v11_cm = plt.imread(v11_cm_path)
    axes[0].imshow(img_v11_cm)
    axes[0].set_title('YOLOv11 Confusion Matrix', fontsize=14, fontweight='bold')
    axes[0].axis('off')
else:
    axes[0].text(0.5, 0.5, 'Confusion matrix not found', 
                ha='center', va='center', transform=axes[0].transAxes, fontsize=12)
    axes[0].set_title('YOLOv11 Confusion Matrix', fontsize=14, fontweight='bold')

# YOLOv12 confusion matrix
v12_runs = sorted(glob.glob('/content/runs/detect/yolov12_*'), key=os.path.getmtime, reverse=True)
v12_cm_path = None
if v12_runs:
    for run_dir in v12_runs:
        cm_path = os.path.join(run_dir, 'confusion_matrix.png')
        if os.path.exists(cm_path):
            v12_cm_path = cm_path
            break

if v12_cm_path and os.path.exists(v12_cm_path):
    img_v12_cm = plt.imread(v12_cm_path)
    axes[1].imshow(img_v12_cm)
    axes[1].set_title('YOLOv12 Confusion Matrix', fontsize=14, fontweight='bold')
    axes[1].axis('off')
else:
    axes[1].text(0.5, 0.5, 'Confusion matrix not found', 
                ha='center', va='center', transform=axes[1].transAxes, fontsize=12)
    axes[1].set_title('YOLOv12 Confusion Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{RESULTS_DIR}/confusion_matrices_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n Saved: {RESULTS_DIR}/confusion_matrices_comparison.png")

## Inference and Visualization

In [ ]:
# Run inference on sample validation images
val_images_path = Path(dataset_path) / 'valid' / 'images'
val_images = list(val_images_path.glob('*.jpg')) + list(val_images_path.glob('*.png'))
sample_test_images = np.random.choice(val_images, min(3, len(val_images)), replace=False)

for idx, img_path in enumerate(sample_test_images):
    print(f"\nProcessing: {img_path.name}")
    
    # Get predictions
    results_v11_inf = model_v11.predict(str(img_path), conf=0.25, save=False, verbose=False)
    results_v12_inf = model_v12.predict(str(img_path), conf=0.25, save=False, verbose=False)
    
    # Visualize
    fig, axes = plt.subplots(1, 3, figsize=(20, 7))
    
    # Original
    original_img = cv2.imread(str(img_path))
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    axes[0].imshow(original_img)
    axes[0].set_title('Original Image', fontsize=13, fontweight='bold')
    axes[0].axis('off')
    
    # YOLOv11
    img_v11 = results_v11_inf[0].plot()
    img_v11 = cv2.cvtColor(img_v11, cv2.COLOR_BGR2RGB)
    axes[1].imshow(img_v11)
    axes[1].set_title(f'YOLOv11 ({len(results_v11_inf[0].boxes)} detections)',
                     fontsize=13, fontweight='bold', color='#3498db')
    axes[1].axis('off')
    
    # YOLOv12
    img_v12 = results_v12_inf[0].plot()
    img_v12 = cv2.cvtColor(img_v12, cv2.COLOR_BGR2RGB)
    axes[2].imshow(img_v12)
    axes[2].set_title(f'YOLOv12 ({len(results_v12_inf[0].boxes)} detections)',
                     fontsize=13, fontweight='bold', color='#e74c3c')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/inference_comparison_{idx+1}.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f" Saved: {RESULTS_DIR}/inference_comparison_{idx+1}.png")

## Summary and Save Results

In [ ]:
# Calculate total execution time
total_execution_time = time.time() - execution_start_time

# Create comprehensive summary
summary = f"""
{'='*80}
CAR DETECTION - COMPREHENSIVE SUMMARY (GOOGLE COLAB)
{'='*80}

DATASET INFORMATION:
  Name: Car Detection Dataset
  Number of Classes: {data_config['nc']}
  Classes: {', '.join([str(c) for c in data_config['names']])}
  Training Images: {len(train_stats['images'])}
  Validation Images: {len(val_stats['images'])}

COMPUTING ENVIRONMENT:
  Platform: Google Colab
  Device: {'GPU - ' + torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}
  PyTorch Version: {torch.__version__}
  CUDA Available: {torch.cuda.is_available()}
{('  CUDA Version: ' + str(torch.version.cuda)) if torch.cuda.is_available() else ''}

YOLOV11 PERFORMANCE:
  mAP@0.5      : {val_results_v11.box.map50:.4f}
  mAP@0.5:0.95 : {val_results_v11.box.map:.4f}
  Precision    : {val_results_v11.box.mp:.4f}
  Recall       : {val_results_v11.box.mr:.4f}
  Training Time: {yolov11_training_time/60:.2f} minutes

YOLOV12 PERFORMANCE:
  mAP@0.5      : {val_results_v12.box.map50:.4f}
  mAP@0.5:0.95 : {val_results_v12.box.map:.4f}
  Precision    : {val_results_v12.box.mp:.4f}
  Recall       : {val_results_v12.box.mr:.4f}
  Training Time: {yolov12_training_time/60:.2f} minutes

PERFORMANCE DIFFERENCE:
  mAP@0.5 Difference: {(val_results_v12.box.map50 - val_results_v11.box.map50):+.4f} ({'higher' if val_results_v12.box.map50 > val_results_v11.box.map50 else 'lower'} for YOLOv12)
  mAP@0.5:0.95 Difference: {(val_results_v12.box.map - val_results_v11.box.map):+.4f} ({'higher' if val_results_v12.box.map > val_results_v11.box.map else 'lower'} for YOLOv12)
  Training Time Difference: {abs(yolov12_training_time - yolov11_training_time)/60:.2f} minutes ({'faster' if yolov12_training_time < yolov11_training_time else 'slower'} for YOLOv12)

TOTAL EXECUTION TIME: {total_execution_time/60:.2f} minutes
DATE: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
{'='*80}
"""

print(summary)

# Save summary
with open(f'{RESULTS_DIR}/colab_execution_summary.txt', 'w') as f:
    f.write(summary)

print(f"\n Saved: {RESULTS_DIR}/colab_execution_summary.txt")

In [ ]:
# Copy all results to Google Drive
print("\n" + "="*80)
print("COPYING RESULTS TO GOOGLE DRIVE")
print("="*80)

try:
    # Copy results directory
    if os.path.exists(RESULTS_DIR):
        drive_results_path = os.path.join(DRIVE_RESULTS, f"results_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
        shutil.copytree(RESULTS_DIR, drive_results_path)
        print(f" Results copied to: {drive_results_path}")
    
    # Copy trained models
    models_drive_path = os.path.join(DRIVE_RESULTS, 'models')
    os.makedirs(models_drive_path, exist_ok=True)
    
    v11_model_path = '/content/runs/detect/yolov11_car_detection/weights/best.pt'
    v12_model_path = '/content/runs/detect/yolov12_car_detection/weights/best.pt'
    
    if os.path.exists(v11_model_path):
        shutil.copy(v11_model_path, os.path.join(models_drive_path, 'yolov11_best.pt'))
        print(" YOLOv11 model saved to Drive")
    
    if os.path.exists(v12_model_path):
        shutil.copy(v12_model_path, os.path.join(models_drive_path, 'yolov12_best.pt'))
        print(" YOLOv12 model saved to Drive")
    
    print(f"\n All results saved to Google Drive: {DRIVE_RESULTS}")
    
except Exception as e:
    print(f"\n  Error copying to Drive: {e}")
    print("Results are still available in /content/results/")

print("="*80)

In [ ]:
# Final summary of generated files
print("\n" + "="*80)
print("GENERATED FILES SUMMARY")
print("="*80)
print(f"\nLocal Results Directory: {RESULTS_DIR}/")
print(f"Google Drive Backup: {DRIVE_RESULTS}/")
print("\nVisualizations:")
print("   eda_class_distribution.png")
print("   sample_annotated_images.png")
print("   performance_comparison_colab.png")
print("   confusion_matrices_comparison.png")
print("   inference_comparison_*.png")
print("\nData Files:")
print("   model_comparison_colab.csv")
print("   colab_execution_summary.txt")
print("\nTrained Models:")
print("   /content/runs/detect/yolov11_car_detection/weights/best.pt")
print("   /content/runs/detect/yolov12_car_detection/weights/best.pt")
print("\n" + "="*80)
print(" ALL TASKS COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\nTotal execution time: {total_execution_time/60:.2f} minutes")
print(f"Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")